In [26]:
!pip install xlsxwriter
import numpy as np
import pandas as pd
import matplotlib as mpl
import math
import xlsxwriter     
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_rows', None)
pd.set_option("display.precision", 10)

You should consider upgrading via the 'C:\Users\trifo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [27]:
#---------------------------------------------------------------CV Stuff---------------------------------------------------------------#
############ FILL IN the 'PotentialConstant' below for CV Files (RHE Value) ############
PotentialConstant = 0.226
###########

Area = 2.75 * 2.75 * math.pi / 100
Beginning = 0.4
End = 0.2
DeltaV = Beginning - End

############ FILL IN the 'FileList' below with all CV file names comma-separated like current format ############
FileList = ['2_FeAc_NC-ML-9_big_batch_121521 cond CV-1.txt', '2_FeAc_NC-ML-9_big_batch_121521 cond CV-2.txt', 
'2_FeAc_NC-ML-9_big_batch_121521 cond CV-3.txt', '2_FeAc_NC-ML-9_big_batch_121521 CV aft ORR.txt']
#All files will be plotted against each other
#Last two files will be plotted against each other
#Last two files will have the integral calculated
###########

Range = len(FileList)
DataFrameListBeforeTreatment = []
DataFrameList = []
ScanRateList = []
ForwardCapacitanceList = []
BackwardCapacitanceList = []
TotalChargeList = []
TotalCapacitanceList = []
CycleList = []
TableList = []

for file in FileList:
    df = pd.read_csv(file)
    DataFrameList.append(df)

    for rowidx in range(df.shape[0]):
        if ''.join(x for x in df.iloc[rowidx, 0] if not x.isdigit() and x not in '.').strip() == "Scan Rate (V/s) =":
            ScanRateList.append(float(''.join(x for x in df.iloc[rowidx, 0] if x.isdigit() or x in '.')))
            break

    for rowidx in range(df.shape[0]):
        if ''.join(x for x in df.iloc[rowidx, 0] if not x.isdigit()).strip() == "Segment =":
            CycleList.append(float(''.join(x for x in df.iloc[rowidx, 0] if x.isdigit())))
            break

    CycleEndValue = df.iloc[-1, 0]
    for rowidx in range(df.shape[0] - 2, -1, -1):
        if df.iloc[rowidx, 0] == CycleEndValue:
            CycleBeginning = rowidx
            break
    df.drop(df.index[0:CycleBeginning - 1], inplace = True)
    df.reset_index(drop=True, inplace = True)
    df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0])
    df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1])
    df.columns = ['Potential/V', 'Current/I']
    DataFrameListBeforeTreatment.append(df.copy(deep = True))
    df.columns = ['Potential/V vs RHE', 'Current Density/mA/cm^2']
    df.iloc[:, 0] += PotentialConstant
    df.iloc[:, 0] = df.iloc[:, 0].round(3)
    df.iloc[:, 1] = df.iloc[:, 1] * 1000 / Area


#CycleList
#Capacitance Calculation

for index, dataframe in enumerate(DataFrameList):
    forwardcapacitance = 0
    for rowidx in range(dataframe.shape[0]):
        if CycleList[index] % 2 == 0:
            if dataframe.iloc[rowidx, 0] <= Beginning:
                forwardcapacitance += (dataframe.iloc[rowidx - 2, 0] - dataframe.iloc[rowidx - 1, 0]) * (dataframe.iloc[rowidx - 2, 1] + dataframe.iloc[rowidx - 1, 1]) * Area / (2 * ScanRateList[index])
                if dataframe.iloc[rowidx, 0] <= End:
                    break
        if CycleList[index] % 2 == 1:
            if dataframe.iloc[rowidx, 0] >= End:
                forwardcapacitance += (dataframe.iloc[rowidx - 2, 0] - dataframe.iloc[rowidx - 1, 0]) * (dataframe.iloc[rowidx - 2, 1] + dataframe.iloc[rowidx - 1, 1]) * Area / (2 * ScanRateList[index])
                if dataframe.iloc[rowidx, 0] >= Beginning:
                    break
    ForwardCapacitanceList.append(forwardcapacitance)

for index, dataframe in enumerate(DataFrameList):
    backwardcapacitance = 0
    for rowidx in range(dataframe.shape[0] - 1, -1, -1):
        if CycleList[index] % 2 == 0:
            if dataframe.iloc[rowidx, 0] <= Beginning:
                backwardcapacitance += (dataframe.iloc[rowidx - 2, 0] - dataframe.iloc[rowidx - 1, 0]) * (dataframe.iloc[rowidx - 2, 1] + dataframe.iloc[rowidx - 1, 1]) * Area / (2 * ScanRateList[index])
                if dataframe.iloc[rowidx, 0] <= End:
                    break
        if CycleList[index] % 2 == 1:
            if dataframe.iloc[rowidx, 0] >= End:
                backwardcapacitance += (dataframe.iloc[rowidx - 2, 0] - dataframe.iloc[rowidx - 1, 0]) * (dataframe.iloc[rowidx - 2, 1] + dataframe.iloc[rowidx - 1, 1]) * Area / (2 * ScanRateList[index])
                if dataframe.iloc[rowidx, 0] >= Beginning:
                    break
    BackwardCapacitanceList.append(backwardcapacitance)

for count, idx in enumerate(range(Range)):
    TotalCapacitanceList.append(abs((ForwardCapacitanceList[idx] + BackwardCapacitanceList[idx]) / (2 * DeltaV)))
    TotalChargeList.append(abs(ForwardCapacitanceList[idx] + BackwardCapacitanceList[idx]))

#---------------------------------------------------------------Integral Calculation Portion---------------------------------------------------------------#
############ FILL IN starting and ending x-value of integration for the TOP (from left to right of x-axis) ############
TopStartingIntegrationXValue = 0.4
TopEndingIntegrationXValue = 0.8
############

TopCurveCalculation = []
TopLineCalculation = []
TopIntegralFinalValues = []

############ FILL IN starting and ending x-value of integration for the BOTTOM (from left to right of x-axis) ############
BottomStartingIntegrationXValue = 0.4
BottomEndingIntegrationXValue = 0.8
###########

BottomCurveCalculation = []
BottomLineCalculation = []
BottomIntegralFinalValues = []

#Top Integral Calculation
for idx in range(len(FileList) - 2, len(FileList)):
    TopCurveValue = 0
    TopLineValue = 0
    TopStartRowIndex = None
    TopEndRowIndex = None
    normaldf = DataFrameList[idx]
    modifieddf = normaldf.copy(deep = True)

    for rowidx in range(df.shape[0]):
        if (normaldf.iloc[rowidx, 0] == TopStartingIntegrationXValue):
            TopStartRowIndex = rowidx
            break
    for rowidx in range(df.shape[0]):
        if (normaldf.iloc[rowidx, 0] == TopEndingIntegrationXValue):
            TopEndRowIndex = rowidx
            break

    slope = (normaldf.iloc[TopEndRowIndex, 1] - normaldf.iloc[TopStartRowIndex, 1]) / (normaldf.iloc[TopEndRowIndex, 0] - normaldf.iloc[TopStartRowIndex, 0])
    yintercept = normaldf.iloc[TopEndRowIndex, 1] - (slope * normaldf.iloc[TopEndRowIndex, 0])
    for rowidx in range(TopStartRowIndex, TopEndRowIndex + 1):
        modifieddf.iloc[rowidx, 1] = (slope * normaldf.iloc[rowidx, 0]) + yintercept

    for rowidx in range(normaldf.shape[0]):
        if normaldf.iloc[rowidx, 0] >= TopStartingIntegrationXValue:
            TopCurveValue += (normaldf.iloc[rowidx - 2, 0] - normaldf.iloc[rowidx - 1, 0]) * (normaldf.iloc[rowidx - 2, 1] + normaldf.iloc[rowidx - 1, 1]) * Area / (2 * ScanRateList[idx])
            if normaldf.iloc[rowidx, 0] >= TopEndingIntegrationXValue:
                break
    TopCurveCalculation.append(abs(TopCurveValue))

    for rowidx in range(modifieddf.shape[0]):
        if modifieddf.iloc[rowidx, 0] >= TopStartingIntegrationXValue:
            TopLineValue += (modifieddf.iloc[rowidx - 2, 0] - modifieddf.iloc[rowidx - 1, 0]) * (modifieddf.iloc[rowidx - 2, 1] + modifieddf.iloc[rowidx - 1, 1]) * Area / (2 * ScanRateList[idx])
            if modifieddf.iloc[rowidx, 0] >= TopEndingIntegrationXValue:
                break
    TopLineCalculation.append(abs(TopLineValue))

#Bottom Integral Calculation
for idx in range(len(FileList) - 2, len(FileList)):
    BottomCurveValue = 0
    BottomLineValue = 0
    BottomStartRowIndex = None
    BottomEndRowIndex = None
    regulardf = DataFrameList[idx]
    altereddf = regulardf.copy(deep=True)

    for rowidx in range(normaldf.shape[0] - 1, -1, -1):
        if regulardf.iloc[rowidx, 0] == BottomStartingIntegrationXValue:
            BottomStartRowIndex = rowidx
            break
    
    for rowidx in range(normaldf.shape[0] - 1, -1, -1):
        if regulardf.iloc[rowidx, 0] == BottomEndingIntegrationXValue:
            BottomEndRowIndex = rowidx
            break
    
    otherslope = (regulardf.iloc[BottomEndRowIndex, 1] - regulardf.iloc[BottomStartRowIndex, 1]) / (regulardf.iloc[BottomEndRowIndex, 0] - regulardf.iloc[BottomStartRowIndex, 0])
    otheryintercept = regulardf.iloc[BottomEndRowIndex, 1] - (otherslope * regulardf.iloc[BottomEndRowIndex, 0])

    for rowidx in range(BottomStartRowIndex, BottomEndRowIndex - 1, -1):
        altereddf.iloc[rowidx, 1] = (otherslope * regulardf.iloc[rowidx, 0]) + otheryintercept

    for rowidx in range(regulardf.shape[0] - 1, -1, -1):
        if regulardf.iloc[rowidx, 0] >= BottomStartingIntegrationXValue:
            BottomCurveValue += (regulardf.iloc[rowidx - 2, 0] - regulardf.iloc[rowidx - 1, 0]) * (regulardf.iloc[rowidx - 2, 1] + regulardf.iloc[rowidx - 1, 1]) * Area / (2 * ScanRateList[idx])
            if regulardf.iloc[rowidx, 0] >= BottomEndingIntegrationXValue:
                
                break
    BottomCurveCalculation.append(abs(BottomCurveValue))

    for rowidx in range(altereddf.shape[0] - 1, -1, -1):
        if altereddf.iloc[rowidx, 0] >= BottomStartingIntegrationXValue:
            BottomLineValue += (altereddf.iloc[rowidx - 2, 0] - altereddf.iloc[rowidx - 1, 0]) * (altereddf.iloc[rowidx - 2, 1] + altereddf.iloc[rowidx - 1, 1]) * Area / (2 * ScanRateList[idx])
            if altereddf.iloc[rowidx, 0] >= BottomEndingIntegrationXValue:
                break
    BottomLineCalculation.append(abs(BottomLineValue))
    
  

for idx in range(0, 2):
    TopIntegralFinalValues.append(abs(TopCurveCalculation[idx] - TopLineCalculation[idx]))
    BottomIntegralFinalValues.append(abs(BottomCurveCalculation[idx] - BottomLineCalculation[idx]))

[8.3005375, 8.986234375000004, 8.500733749999995, 10.618086250000003]

[3.3202150000000006, 3.5944937500000016, 3.4002934999999983, 4.247234500000001]

In [28]:
#Outputting graphs and plots to Excel file
ExcelFileName = FileList[0][0:-14] + '.xlsx'
SheetName = 'CV Data'
writer = pd.ExcelWriter(ExcelFileName, engine='xlsxwriter')
col = 0
for idx in range(len(FileList)):
    DataFrameListBeforeTreatment[idx].to_excel(writer, sheet_name = SheetName, index = False, startrow = 1, startcol = col)
    col += len(DataFrameListBeforeTreatment[idx].columns)
    DataFrameList[idx].to_excel(writer, sheet_name = SheetName, index = False, startrow = 1, startcol = col)
    col += len(DataFrameListBeforeTreatment[idx].columns) + 2

Legend = []
for file in FileList:
    name = file[0:len(file) - 4]
    Legend.append(name)
worksheet = writer.sheets[SheetName]
column = 0
for idx in range(len(FileList)):
    worksheet.write_string(0, column, Legend[idx])
    column += len(DataFrameListBeforeTreatment[idx].columns) + len(DataFrameListBeforeTreatment[idx].columns) + 2

workbook = writer.book
chart = workbook.add_chart({'type': 'scatter', 'subtype': 'smooth'})
namecol = 0
categorycol = 2
valuescol = 3
for idx in range(len(FileList)):
    chart.add_series({
        'name': [SheetName, 0, namecol, 0, namecol],
        'categories': [SheetName, 2, categorycol, 2 + len(DataFrameList[idx]) - 1, categorycol],
        'values': [SheetName, 2,  valuescol, 2 + len(DataFrameList[idx]) - 1, valuescol],
    })
    namecol += 6
    categorycol += 6
    valuescol += 6
chart.set_title({'name': 'All CVs'})
chart.set_x_axis({'min': 0, 'max': 1, 'name': 'Potential, V vs RHE'})
chart.set_y_axis({'name': 'Current Density, mA/cm^2'})
worksheet.insert_chart(0, col, chart)

othernamecol = col - 12
othercategorycol = col - 10
othervaluescol = col - 9
otherchart = workbook.add_chart({'type': 'scatter', 'subtype': 'smooth'})
for idx in range(len(FileList) - 2, len(FileList)):
    otherchart.add_series({
        'name': [SheetName, 0, othernamecol, 0, othernamecol],
        'categories': [SheetName, 2, othercategorycol, 2 + len(DataFrameList[idx]) - 1, othercategorycol],
        'values': [SheetName, 2,  othervaluescol, 2 + len(DataFrameList[idx]) - 1, othervaluescol],
    })
    othernamecol += 6
    othercategorycol += 6
    othervaluescol +=6
otherchart.set_title({'name': 'CV before and after ORR'})
otherchart.set_x_axis({'min': 0, 'max': 1, 'name': 'Potential, V vs RHE'})
otherchart.set_y_axis({'name': 'Current Density, mA/cm^2'})
worksheet.insert_chart(15, col, otherchart)

DeltaVList = []
TableHeader = []
for idx in range(len(FileList)):
    ForwardCapacitanceList[idx] = abs(ForwardCapacitanceList[idx])
    BackwardCapacitanceList[idx] = abs(BackwardCapacitanceList[idx])
    DeltaVList.append(DeltaV)
    TableHeader.append(Legend[idx][32:])
TableList = [ScanRateList, DeltaVList, ForwardCapacitanceList, BackwardCapacitanceList, TotalChargeList, TotalCapacitanceList]
dftable = pd.DataFrame(TableList)
dftable.columns = TableHeader
dftable.index = ['Scan Rate (V/s)', 'Delta V (V)', 'Double Layer Charge Pos (mA)', 'Double Layer Charge Neg (mA)', 'Total Charge (mA)', 'Capacitance (mF)']
dftable.to_excel(writer, sheet_name = SheetName, startrow = 30, startcol = col)

OtherTableHeader = []
for idx in range(len(FileList) - 2, len(FileList)):
    OtherTableHeader.append(Legend[idx][32:])
OtherTableList = [TopIntegralFinalValues, BottomIntegralFinalValues]
otherdftable = pd.DataFrame(OtherTableList)
otherdftable.columns = OtherTableHeader
otherdftable.index = ['Top Charge (mCs)', 'Bottom Charge (mCs)']
otherdftable.to_excel(writer, sheet_name = SheetName, startrow = 38, startcol = col)

######REMOVE THIS LATER WHEN YOU PASTE ORR STUFF BACK IN
writer.save()

0

0

0

0

0

0